# Import packages and database

In [61]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.segmentation as seg
from biogeme.expressions import Beta, Variable, Derive
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
from collections import namedtuple

data_file ='https://raw.githubusercontent.com/GustavePellier/MMOB/main/lpmc19.dat'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC

database = db.Database('LPMC', LPMC)
all_results = {}

# Model 0

We calculate the total public transport duration and the total driving cost

In [62]:
LPMC["dur_pt"]= LPMC["dur_pt_access"] + LPMC["dur_pt_rail"] + LPMC["dur_pt_bus"] + LPMC["dur_pt_int"] 
LPMC["cost_drive"] = LPMC["cost_driving_ccharge"] + LPMC["cost_driving_fuel"]

Some variables are created with the columns that seem to be useful

In [63]:
travel_mode=Variable('travel_mode')

dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')

pt_interchanges=Variable('pt_interchanges')

cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
cost_drive=Variable('cost_drive')
cost_pt=Variable('cost_transit')

time_walk=Variable('dur_walking')
time_cycle=Variable('dur_cycling')
time_pt=Variable('dur_pt')
time_drive=Variable('dur_driving')

female=Variable('female')
age=Variable('age')

There are 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [64]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

Utility functions creation

In [65]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

availability of each mode, all available here

In [66]:
av = {1: 1, 2: 1, 3: 1, 4:1}

The estimation results (parameter values, t-tests or p-values, null and final log likelihoods)

In [67]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.878619,0.107672,-36.022418,0.0
asc_drive,-1.295191,0.080505,-16.088354,0.0
asc_pt,-0.503354,0.054123,-9.300119,0.0
beta_cost,-0.193629,0.013958,-13.871889,0.0
beta_time,-5.495527,0.208596,-26.345342,0.0


In [68]:
res.compile_estimation_results(all_results)

(                                         Model0
 Number of estimated parameters                5
 Sample size                                5000
 Final log likelihood               -4587.818071
 Akaike Information Criterion        9185.636142
 Bayesian Information Criterion      9218.222108
 asc_cycle (t-test)                 -3.88  (-36)
 asc_drive (t-test)                -1.3  (-16.1)
 asc_pt (t-test)                  -0.503  (-9.3)
 beta_cost (t-test)              -0.194  (-13.9)
 beta_time (t-test)                -5.5  (-26.3),
 {'Model0': 'Model0'})

In [69]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1

We choose to do a specification for the time attribute, the cost stays generic in this model. Therefore we obtain a beta_time parameter for each transportation mode. 

In [70]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)

The utility functions are rewritten with the new parameters :

In [71]:
v_walk_model1= beta_time_walk * time_walk  
v_cycle_model1= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1= asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1= asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

The results are computed (parameter values, t-tests or p-values, null and final log likelihoods)

In [72]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model_1'
all_results['Model1'] = biogeme_model1.estimate()
results_specific_time = biogeme_model1.estimate()
results_specific_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.847224,0.202570,-23.928654,0.0
asc_drive,-1.969873,0.137929,-14.281808,0.0
asc_pt,-2.337196,0.139506,-16.753342,0.0
beta_cost,-0.182008,0.016087,-11.314047,0.0
beta_time_cycle,-5.312452,0.462002,-11.498762,0.0
beta_time_drive,-6.474307,0.378989,-17.083106,0.0
beta_time_pt,-3.538619,0.250656,-14.117426,0.0
beta_time_walk,-8.430765,0.418990,-20.121621,0.0


In [73]:
res.compile_estimation_results(all_results)

(                                         Model0           Model1
 Number of estimated parameters                5                8
 Sample size                                5000             5000
 Final log likelihood               -4587.818071     -4275.496542
 Akaike Information Criterion        9185.636142      8566.993084
 Bayesian Information Criterion      9218.222108       8619.13063
 asc_cycle (t-test)                 -3.88  (-36)   -4.85  (-23.9)
 asc_drive (t-test)                -1.3  (-16.1)   -1.97  (-14.3)
 asc_pt (t-test)                  -0.503  (-9.3)   -2.34  (-16.8)
 beta_cost (t-test)              -0.194  (-13.9)  -0.182  (-11.3)
 beta_time (t-test)                -5.5  (-26.3)                 
 beta_time_cycle (t-test)                          -5.31  (-11.5)
 beta_time_drive (t-test)                          -6.47  (-17.1)
 beta_time_pt (t-test)                             -3.54  (-14.1)
 beta_time_walk (t-test)                           -8.43  (-20.1),
 {'Model0

In [74]:
biogeme_model1.calculateNullLoglikelihood(av)

-6931.471805599917

We compare the 2 models with the likelihood ratio test because model 0 is a reduction of model 1.  

In [75]:
results_specific_time.likelihood_ratio_test(results_generic, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=624.6430583499878, threshold=7.814727903251179)

# Model 2

Using Model 1 as the base model, we choose to study how gender interact with the
ASCs and the alternative attributes. 

Lets start by segmenting the population by gender

In [76]:
gender_segmentation = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

### ASC segmentation

In [77]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
segmented_asc_cycle = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt = Beta('asc_pt', 0, None, None, 0)
segmented_asc_pt = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive = Beta('asc_drive', 0, None, None, 0)
segmented_asc_drive = seg.Segmentation(asc_drive,[gender_segmentation]).segmented_beta()

We redefine the value function of the 4 alternatives, introducing the segmented ASC and the segmented cost attribute

In [78]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

We can compute the result of the new model

In [79]:
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model2'] = biogeme_model2.estimate()
results_segmented_gender = biogeme_model2.estimate()
results_segmented_gender.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.454910,0.253347,-21.531354,0.000000e+00
asc_cycle_male,1.051802,0.208766,5.038188,4.699608e-07
asc_drive,-1.893661,0.144300,-13.123043,0.000000e+00
asc_drive_male,-0.197962,0.095253,-2.078277,3.768381e-02
asc_pt,-2.208490,0.146370,-15.088352,0.000000e+00
asc_pt_male,-0.333165,0.099554,-3.346591,8.181180e-04
beta_cost,-0.182827,0.016154,-11.317822,0.000000e+00
beta_time_cycle,-5.580780,0.473118,-11.795745,0.000000e+00
beta_time_drive,-6.487957,0.380056,-17.071074,0.000000e+00
beta_time_pt,-3.527004,0.251404,-14.029204,0.000000e+00


In [80]:
biogeme_model2.calculateNullLoglikelihood(av)

-6931.471805599917

### Cost segmentation

And its interaction on the cost attribute in the car and public transport alternative.

In [81]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
segmented_beta_cost = seg.Segmentation(beta_cost, [gender_segmentation]).segmented_beta()

In [82]:
v_walk_model2_2= beta_time_walk * time_walk  
v_cycle_model2_2= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2_2= asc_pt + beta_time_pt * time_pt + segmented_beta_cost * cost_pt
v_drive_model2_2= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

We can compute the result of the new model

In [83]:
V_model2_2 = {1: v_walk_model2_2 , 2: v_cycle_model2_2, 3: v_pt_model2_2, 4: v_drive_model2_2}
logprob_model2_2 = models.loglogit(V_model2_2, av, travel_mode)
biogeme_model2_2 = bio.BIOGEME(database, logprob_model2_2)
biogeme_model2_2.modelName = 'Model_2_2'
all_results['Model2_2'] = biogeme_model2_2.estimate()
results_segmented_gender_2 = biogeme_model2_2.estimate()
results_segmented_gender_2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.843766,0.202192,-23.956237,0.000000e+00
asc_drive,-1.971716,0.137731,-14.315753,0.000000e+00
asc_pt,-2.343113,0.139307,-16.819783,0.000000e+00
beta_cost,-0.151590,0.018749,-8.085464,6.661338e-16
beta_cost_male,-0.065393,0.031840,-2.053794,3.999566e-02
beta_time_cycle,-5.348638,0.459249,-11.646498,0.000000e+00
beta_time_drive,-6.483025,0.380413,-17.042049,0.000000e+00
beta_time_pt,-3.536891,0.250704,-14.107830,0.000000e+00
beta_time_walk,-8.435602,0.418400,-20.161583,0.000000e+00


In [84]:
biogeme_model2_2.calculateNullLoglikelihood(av)

-6931.471805599917

### Compiling all results from the different models

In [85]:
res.compile_estimation_results(all_results)

(                                         Model0           Model1  \
 Number of estimated parameters                5                8   
 Sample size                                5000             5000   
 Final log likelihood               -4587.818071     -4275.496542   
 Akaike Information Criterion        9185.636142      8566.993084   
 Bayesian Information Criterion      9218.222108       8619.13063   
 asc_cycle (t-test)                 -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                  -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)              -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                -5.5  (-26.3)                    
 beta_time_cycle (t-test)                          -5.31  (-11.5)   
 beta_time_drive (t-test)                          -6.47  (-17.1)   
 beta_time_pt (t-test)                             -3.54  (-14.1)   
 beta_time_walk (t-test)          

### Likelihood ratio test to check if model_2 and model_2_2 are equivalent or not

In [86]:
results_segmented_gender.likelihood_ratio_test(results_segmented_gender_2, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=55.92757888516644, threshold=5.991464547107979)

Model_2 is preferred because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2

### Comparison with model 1

In [87]:
results_segmented_gender.likelihood_ratio_test(results_specific_time, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=60.51262152961135, threshold=7.814727903251179)

Model_2 is preferred to Model_1 because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2.

# Model 3 - Thomas

In [88]:
# Repartir du model segmenté, Modèle 2

In [89]:
#GENDER segmentation
gender_segmentation_3bis = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

In [90]:
asc_cycle_3bis = Beta('asc_cycle_3bis', 0, None, None, 0)
segmented_asc_cycle_3bis = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt_3bis = Beta('asc_pt_3bis', 0, None, None, 0)
segmented_asc_pt_3bis = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive_3bis = Beta('asc_drive_3bis', 0, None, None, 0)
segmented_asc_drive_3bis = seg.Segmentation(asc_drive_3bis,[gender_segmentation_3bis]).segmented_beta()

In [91]:
beta_time_walk_3bis = Beta('beta_time_walk_3bis', 0, None, None, 0)
beta_time_cycle_3bis = Beta('beta_time_cycle_3bis', 0, None, None, 0)

beta_time_pt_3bis = Beta('beta_time_pt_3bis', 0, None, None, 0)
beta_time_pt_3bis_squarred = Beta('beta_time_pt_3bis_squarred', 0, None, None, 0)

beta_time_drive_3bis = Beta('beta_time_drive_3bis', 0, None, None, 0)
beta_cost_3bis = Beta('beta_cost_3bis', 0, None, None, 0)


v_walk_model_3bis= beta_time_walk_3bis * time_walk  

v_cycle_model_3bis= segmented_asc_cycle_3bis + beta_time_cycle_3bis * time_cycle 

v_pt_model_3bis= segmented_asc_pt_3bis + beta_time_pt_3bis * time_pt + beta_time_pt_3bis_squarred *time_pt*time_pt + beta_cost_3bis * cost_pt

v_drive_model_3bis= segmented_asc_drive_3bis + beta_time_drive_3bis * time_drive + beta_cost_3bis * cost_drive

In [92]:
V_model_3bis = {1: v_walk_model_3bis , 2: v_cycle_model_3bis, 3: v_pt_model_3bis, 4: v_drive_model_3bis}
logprob_model_3bis = models.loglogit(V_model_3bis, av, travel_mode)
biogeme_model_3bis = bio.BIOGEME(database, logprob_model_3bis)
biogeme_model_3bis.modelName = 'Model_3bis'
all_results['Model_3bis'] = biogeme_model_3bis.estimate()
results_segmented_gender_3bis = biogeme_model_3bis.estimate()
results_segmented_gender_3bis.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.373633,0.250689,-21.435415,0.000000e+00
asc_cycle_male,1.052224,0.208454,5.047748,4.470479e-07
asc_drive_3bis,-1.802140,0.145490,-12.386675,0.000000e+00
asc_drive_3bis_male,-0.198042,0.094447,-2.096845,3.600733e-02
asc_pt,-2.490305,0.160124,-15.552319,0.000000e+00
asc_pt_male,-0.339756,0.101136,-3.359391,7.811442e-04
beta_cost_3bis,-0.180475,0.016392,-11.010049,0.000000e+00
beta_time_cycle_3bis,-5.573343,0.458404,-12.158134,0.000000e+00
beta_time_drive_3bis,-6.562530,0.388238,-16.903348,0.000000e+00
beta_time_pt_3bis,-1.983146,0.470332,-4.216483,2.481418e-05


In [93]:
#COST segmentation

In [94]:
beta_cost_33 = Beta('beta_cost_33', 0, None, None, 0)
segmented_beta_cost_33 = seg.Segmentation(beta_cost_33, [gender_segmentation_3bis]).segmented_beta()

In [95]:
beta_time_pt_33_squarred = Beta('beta_time_pt_33_squarred', 0, None, None, 0)

v_walk_model_33= beta_time_walk * time_walk  

v_cycle_model_33= asc_cycle + beta_time_cycle * time_cycle 

v_pt_model_33= asc_pt + beta_time_pt * time_pt + beta_time_pt_33_squarred * time_pt * time_pt + segmented_beta_cost * cost_pt

v_drive_model_33= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

In [96]:
V_model_33 = {1: v_walk_model_33 , 2: v_cycle_model_33, 3: v_pt_model_33, 4: v_drive_model_33}
logprob_model_33 = models.loglogit(V_model_33, av, travel_mode)
biogeme_model_33 = bio.BIOGEME(database, logprob_model_33)
biogeme_model_33.modelName = 'Model__33'
all_results['Model_33'] = biogeme_model_33.estimate()
results_segmented_gender_33 = biogeme_model_33.estimate()
results_segmented_gender_33.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.761755,0.199492,-23.869348,0.000000e+00
asc_drive,-1.881506,0.139071,-13.529130,0.000000e+00
asc_pt,-2.623362,0.154063,-17.027877,0.000000e+00
beta_cost,-0.149036,0.018924,-7.875293,3.330669e-15
beta_cost_male,-0.065963,0.032233,-2.046409,4.071610e-02
beta_time_cycle,-5.345590,0.445806,-11.990847,0.000000e+00
beta_time_drive,-6.555686,0.388303,-16.882896,0.000000e+00
beta_time_pt,-2.013958,0.470791,-4.277818,1.887340e-05
beta_time_pt_33_squarred,-1.219255,0.352239,-3.461446,5.372810e-04
beta_time_walk,-8.288549,0.418766,-19.792813,0.000000e+00


### Comparison model_3bis and model_33

In [97]:


results_segmented_gender_3bis.likelihood_ratio_test(results_segmented_gender_33, 0.05)


LRTuple(message='H0 can be rejected at level 5.0%', statistic=56.37685236132165, threshold=5.991464547107979)

In [98]:
print(results_segmented_gender_3bis.printGeneralStatistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4236.589
Final log likelihood:	-4236.589
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00283
Akaike Information Criterion:	8497.178
Bayesian Information Criterion:	8575.385
Final gradient norm:	2.8424E-04
Nbr of threads:	8



In [99]:
print(results_segmented_gender_33.printGeneralStatistics())

Number of estimated parameters:	10
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4264.778
Final log likelihood:	-4264.778
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00234
Akaike Information Criterion:	8549.555
Bayesian Information Criterion:	8614.727
Final gradient norm:	2.1479E-03
Nbr of threads:	8



### Preferred model is model_3bis

The model_3bis is preferred to model_33 as its log likelihood is greater and the AIC and BIC are smaller as well for this model.

### Comparison model_3bis and model_2

In [100]:
results_segmented_gender_3bis.likelihood_ratio_test(results_segmented_gender, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=17.302101969298747, threshold=3.841458820694124)

In [101]:
print(results_segmented_gender.printGeneralStatistics())

Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4245.24
Final log likelihood:	-4245.24
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00259
Akaike Information Criterion:	8512.48
Bayesian Information Criterion:	8584.17
Final gradient norm:	6.6512E-03
Nbr of threads:	8



We notice that the Final loglikelyhood as well as AIC & BIC are lower for the model_3bis than for model_2.
As those two model are not equivalent at the 5% significance level, we can conclude that the model 3 bis is preferred

### Version ancienne du power series 

In [102]:
# Power Series transformation on 'pt_dur'.

asc_cycle_3 = Beta('asc_cycle_3', 0, None, None, 0)

asc_pt_3 = Beta('asc_pt_3', 0, None, None, 0)

asc_drive_3 = Beta('asc_drive_3', 0, None, None, 0)


beta_time_walk_3 = Beta('beta_time_walk_3', 0, None, None, 0)
beta_time_cycle_3 = Beta('beta_time_cycle_3', 0, None, None, 0)

beta_time_pt_3 = Beta('beta_time_pt_3', 0, None, None, 0)
beta_time_pt_3_squarred = Beta('beta_time_pt_3_squarred', 0, None, None, 0)

beta_time_drive_3 = Beta('beta_time_drive_3', 0, None, None, 0)
beta_cost_3 = Beta('beta_cost_3', 0, None, None, 0)



v_walk_model_3= beta_time_walk_3 * time_walk 

v_cycle_model_3= asc_cycle_3 + beta_time_cycle_3 * time_cycle 

v_pt_model_3= asc_pt_3 + beta_time_pt_3 * time_pt +beta_time_pt_3_squarred * time_pt * time_pt + beta_cost * cost_pt

v_drive_model_3= asc_drive_3 + beta_time_drive_3 * time_drive + beta_cost_3 * cost_drive

In [103]:
V_model_3 = {1: v_walk_model_3 , 2: v_cycle_model_3, 3: v_pt_model_3, 4: v_drive_model_3}

logprob_model_3 = models.loglogit(V_model_3, av, travel_mode)

biogeme_model_3 = bio.BIOGEME(database, logprob_model_3)

biogeme_model_3.modelName = 'Model_3'

all_results['Model_3'] = biogeme_model_3.estimate()

results_power_3 = biogeme_model_3.estimate()

results_power_3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle_3,-4.763434,0.199847,-23.835399,0.000000e+00
asc_drive_3,-1.873748,0.139320,-13.449256,0.000000e+00
asc_pt_3,-2.601264,0.155172,-16.763701,0.000000e+00
beta_cost,-0.210770,0.031140,-6.768429,1.301892e-11
beta_cost_3,-0.168058,0.017600,-9.548949,0.000000e+00
beta_time_cycle_3,-5.431875,0.464148,-11.702905,0.000000e+00
beta_time_drive_3,-6.753131,0.442152,-15.273340,0.000000e+00
beta_time_pt_3,-2.024758,0.471819,-4.291384,1.775633e-05
beta_time_pt_3_squarred,-1.228550,0.352701,-3.483262,4.953435e-04
beta_time_walk_3,-8.319325,0.421317,-19.746000,0.000000e+00


In [104]:
#Calculate Null Log likelihood

biogeme_model_3.calculateNullLoglikelihood(av)

-6931.471805599917

In [105]:
#Comparison to model 1

results_power_3.likelihood_ratio_test(results_specific_time, 0.05)




LRTuple(message='H0 can be rejected at level 5.0%', statistic=18.422522396929708, threshold=5.991464547107979)

The model 3 is not preferred compare to model 1.

-Is it due to the non linear transformation that is not appropriate ? Should we use boxplot instead ? --> Yes we should 

-Is it stated 'non-linear transfronation on one of the variables' so I just took public transport time, should I apply to the other time ? --> I think but I will ask

-Should the non linear transformation be applied to an other time variable like car ? Yes I think but I will ask

-Is the loglikelyhood ratio test still appropirate ? --> Maybe we should use a cox-text instead 

Remarks: I think that the non-linear transformation is not done on beta but rather on variables such as time, cost, etc.!


# Model 3 - Salomé

In [106]:

lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0) #vu dans le modèle 01-logit_airline_solution
boxcox_time_walk = models.boxcox(time_walk, lambda_boxcox)
boxcox_time_cycle = models.boxcox(time_cycle, lambda_boxcox)
boxcox_time_pt = models.boxcox(time_pt, lambda_boxcox)
boxcox_time_drive = models.boxcox(time_drive , lambda_boxcox)
beta_elapsed_time_walk = Beta('beta_elapsed_walk', 0, None, None, 0)
beta_elapsed_time_cycle = Beta('beta_elapsed_cycle', 0, None, None, 0)
beta_elapsed_time_pt = Beta('beta_elapsed_time_pt', 0, None, None, 0)
beta_elapsed_time_drive = Beta('beta_elapsed_time_drive', 0, None, None, 0)


v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive


v_walk_model3 = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model3  = segmented_asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model3  = segmented_asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt 
v_drive_model3  = segmented_asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive 

It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)


In [107]:
V_model3 = {1: v_walk_model3 , 2: v_cycle_model3, 3: v_pt_model3, 4: v_drive_model3}
logprob_model3 = models.loglogit(V_model3, av, travel_mode)
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.modelName = 'Model_3'
all_results['Model_3'] = biogeme_model3.estimate()
results_model3 = biogeme_model3.estimate()
results_model3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.218621,0.303923,-10.590245,0.000000e+00
asc_cycle_male,1.053236,0.210050,5.014205,5.325310e-07
asc_drive,0.068988,0.217652,0.316965,7.512698e-01
asc_drive_male,-0.226070,0.101484,-2.227646,2.590414e-02
asc_pt,1.844831,0.170761,10.803566,0.000000e+00
asc_pt_male,-0.375193,0.107310,-3.496337,4.716931e-04
beta_cost,-0.174891,0.015669,-11.161362,0.000000e+00
beta_elapsed_cycle,-3.318150,0.268401,-12.362676,0.000000e+00
beta_elapsed_time_drive,-3.301160,0.265807,-12.419397,0.000000e+00
beta_elapsed_time_pt,-2.637904,0.182666,-14.441112,0.000000e+00


In [108]:
res.compileEstimationResults(all_results)

The syntax "compileEstimationResults" is deprecated and is replaced by the syntax "compile_estimation_results".


(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496542   
 Akaike Information Criterion             9185.636142      8566.993084   
 Bayesian Information Criterion           9218.222108       8619.13063   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [109]:
print("Null Loglikelihood : ")
biogeme_model3.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

In [110]:
results_model3.likelihood_ratio_test(results_segmented_gender, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=175.9877300601147, threshold=3.841458820694124)

In [111]:
print(results_model3.printGeneralStatistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4157.246
Final log likelihood:	-4157.246
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00289
Akaike Information Criterion:	8338.493
Bayesian Information Criterion:	8416.699
Final gradient norm:	8.8281E-04
Nbr of threads:	8



To compare Model 2 and Model 3 I think we should use a Cox test as Model 3 is not a a linear restriction of Model 2, to do that we create a composite model C such that both models 1 and 2 are restricted cases of model C.


In [112]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive


v_walk_model3 = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model3  = segmented_asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model3  = segmented_asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt 
v_drive_model3  = segmented_asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive 


v_walk_model_generic = beta_time_walk * time_walk + beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model_generic = segmented_asc_cycle + beta_time_cycle * time_cycle +  beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model_generic = segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt + beta_elapsed_time_pt * boxcox_time_pt 
v_drive_model_generic = segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive + beta_elapsed_time_drive * boxcox_time_drive

In [113]:
V_model_generic = {1: v_walk_model_generic , 2: v_cycle_model_generic, 3: v_pt_model_generic, 4: v_drive_model_generic}
logprob_model_generic = models.loglogit(V_model_generic, av, travel_mode)
biogeme_model_generic = bio.BIOGEME(database, logprob_model_generic)
biogeme_model_generic.modelName = 'Model_generic'
all_results['Model_generic'] = biogeme_model_generic.estimate()
results_model_generic = biogeme_model_generic.estimate()
results_model_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.703620,1.211728,-2.231211,2.566717e-02
asc_cycle_male,1.056108,0.209812,5.033580,4.814035e-07
asc_drive,0.712607,0.896057,0.795269,4.264570e-01
asc_drive_male,-0.224654,0.101186,-2.220203,2.640501e-02
asc_pt,5.548916,1.717761,3.230318,1.236526e-03
asc_pt_male,-0.369701,0.108179,-3.417483,6.320305e-04
beta_cost,-0.170996,0.015857,-10.783554,0.000000e+00
beta_elapsed_cycle,-5.482310,2.093502,-2.618727,8.825845e-03
beta_elapsed_time_drive,-5.385735,2.386486,-2.256763,2.402286e-02
beta_elapsed_time_pt,-2.435589,1.026143,-2.373539,1.761854e-02


In [114]:
res.compileEstimationResults(all_results)

The syntax "compileEstimationResults" is deprecated and is replaced by the syntax "compile_estimation_results".


(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496542   
 Akaike Information Criterion             9185.636142      8566.993084   
 Bayesian Information Criterion           9218.222108       8619.13063   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [115]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=191.55145975720916, threshold=11.070497693516351)

In [116]:
model_base=biogeme_model3
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=15.563729697094459, threshold=9.487729036781154)

## Model 4

We chose to separate the modes by nesting walking and cycling as "soft modes" and keeping driving and PT separeted. 

In [117]:
model_base=biogeme_model3
V_nested=V_model3
#create the nests
Mu_Drive =Beta("Mu_Drive",1,0,None,0)
Mu_PT =Beta("Mu_PT",1,0,None,0)
Mu_SM =Beta("Mu_SoftModes",1,0,None,0)
drive= Mu_Drive,[4]
PT=Mu_PT, [3]
SM=Mu_SM, [1,2]
nests_modes=drive, PT, SM

In [118]:
logprob_model_nested = models.lognested(V_nested, av, nests_modes, travel_mode)
biogeme_model_nested = bio.BIOGEME(database, logprob_model_nested)
biogeme_model_nested.modelName = 'nestmodes'
all_results['Nestmodes'] = biogeme_model_nested.estimate()
results_model_nested = biogeme_model_nested.estimate()
results_model_nested.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_Drive,1.000000,1.797693e+308,5.562685e-309,1.000000e+00
Mu_PT,1.000000,2.584659e-15,3.868982e+14,0.000000e+00
Mu_SoftModes,0.603680,7.647243e-02,7.894088e+00,2.886580e-15
asc_cycle,-2.955126,3.748133e-01,-7.884260e+00,3.108624e-15
asc_cycle_male,1.574252,3.494973e-01,4.504333e+00,6.658165e-06
asc_drive,0.294558,2.368810e-01,1.243484e+00,2.136895e-01
asc_drive_male,-0.147821,1.109199e-01,-1.332681e+00,1.826365e-01
asc_pt,2.181106,1.969323e-01,1.107541e+01,0.000000e+00
asc_pt_male,-0.290546,1.165381e-01,-2.493145e+00,1.266173e-02
beta_cost,-0.171546,1.569916e-02,-1.092708e+01,0.000000e+00


The model is rejected as Mu soft modes in < 1

We try another model : vehicles (bikes, cars, PT) and not vehicle (walk) 

In [123]:
Mu_vehicle=Beta("Mu_vehicle",1,0,None,0)
Mu_not_vehicle=Beta("Mu_not_vehicle",1,0,None,0)
vehicle= Mu_vehicle,[2,3,4]
not_vehicle=Mu_not_vehicle,[1]
nests_vehicle=vehicle, not_vehicle


In [124]:
logprob_model_nestedvehicle = models.lognested(V_nested, av, nests_vehicle, travel_mode)
biogeme_model_nestedvehicle = bio.BIOGEME(database, logprob_model_nestedvehicle)
biogeme_model_nestedvehicle.modelName = 'Model_nested_vehicle'
results_model_nestedvehicle = biogeme_model_nestedvehicle.estimate()
all_results['Model_nested_vehicle'] = results_model_nestedvehicle
table_nestedvehicle=results_model_nestedvehicle.getEstimatedParameters()
table_nestedvehicle

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_not_vehicle,1.000000,2.329809e-15,4.292198e+14,0.000000e+00
Mu_vehicle,1.141439,1.710136e-01,6.674552e+00,2.479883e-11
asc_cycle,-2.353217,9.590802e-01,-2.453618e+00,1.414270e-02
asc_cycle_male,0.899900,2.487290e-01,3.617992e+00,2.968973e-04
asc_drive,0.520684,5.210931e-01,9.992150e-01,3.176905e-01
asc_drive_male,-0.221829,1.000797e-01,-2.216522e+00,2.665578e-02
asc_pt,2.075987,2.938340e-01,7.065171e+00,1.604272e-12
asc_pt_male,-0.352449,1.071100e-01,-3.290530e+00,9.999871e-04
beta_cost,-0.153334,2.783686e-02,-5.508319e+00,3.622770e-08
beta_elapsed_cycle,-2.882245,5.248855e-01,-5.491187e+00,3.992416e-08


In [127]:
res.compileEstimationResults(all_results)

The syntax "compileEstimationResults" is deprecated and is replaced by the syntax "compile_estimation_results".


(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496542   
 Akaike Information Criterion             9185.636142      8566.993084   
 Bayesian Information Criterion           9218.222108       8619.13063   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [128]:
results_base=model_base.estimate()
results_model_nestedvehicle.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=0.904545037627031, threshold=5.991464547107979)

So the model pref keeps being Model_3 (Box-cox transformation)

## Market Shares